In [ ]:
# Import packages 
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

from scipy.io import FortranFile
from scipy import interpolate as intr

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import ESMF as E

import importlib
import glob
import copy
import time

import scripGen as SG
import esmfRegrid as erg
import VertRegrid as vrg

# imprt modules in ../Plotting,  """
#sys.path.append('../Plotting/')
sys.path.append('../Utils/')

import MakePressures as MkP

# Reload local packages that are under
# development
importlib.reload( erg )
importlib.reload( vrg )
importlib.reload( SG )
importlib.reload( MkP )

In [ ]:
# Physical Constants
Rgas = 287.0 # J K-1 kg-1

## Naming conventions for variables:

In [ ]:
# aaa_{CAM,ERA} 
# Indicates the immediate provenance of a variable, e.g.,
#      phis_CAM ==> phis from CAM on CAM's grid
#      phis_ERA ==> phis from ERA on ERA's grid
# lower case 'phis' indicates this is ndarray-like 
#
# aaa_{CAM,ERA}_x{ERA,CAM}
# Indicates variable has been remapped horizontall. So, e.g.
#      phis_ERA_xCAM ==> ERA phis remapped horizontally to the CAM grid 
#
# aaa_{CAM,ERA}_xz{ERA,CAM}
# Indicates variable has been remapped horizontally AND vertically. So, e.g.
#      te_ERA_xzCAM ==> ERA temperature remapped horizontally to the CAM horizontal grid 
#                       and then also vertically interpoated to the CAM vertical grid


In [ ]:
year=2022
month=11
day=1
hour0=0
hour1=hour0+5


monStr=str( year ).zfill(4)+str(month).zfill(2)
print(monStr)

ymdh0=str( year ).zfill(4)+str(month).zfill(2)+str(day).zfill(2)+str(hour0).zfill(2)
ymdh1=str( year ).zfill(4)+str(month).zfill(2)+str(day).zfill(2)+str(hour1).zfill(2)
ymdh=ymdh0+'_'+ymdh1

print(ymdh)

In [ ]:
# Get all the met and topo data we will use

bnd_topo = \
'/glade/p/cgd/amp/juliob/bndtopo/latest/ne30pg3_gmted2010_modis_bedmachine_nc3000_Laplace0100_20230105.nc'
dsTopo_CAM=xr.open_dataset( bnd_topo )
phis_CAM = dsTopo_CAM['PHIS'].values

# Read in ERA5 topography
eratopof='/glade/work/juliob/ERA5-proc/ERA5interp/phis/ERA5_phis.nc'
dsTopo_ERA=xr.open_dataset( eratopof )
phis_ERA=dsTopo_ERA['Z_GDS4_SFC'].values



era5dir = "/glade/collections/rda/data/ds633.6/e5.oper.an.ml/"
wrkdir=era5dir+monStr+"/"


spfile= wrkdir + 'e5.oper.an.ml.128_134_sp.regn320sc.'+ymdh+'.nc'
dsPS_ERA  = xr.open_dataset( spfile )
ps_ERA = dsPS_ERA['SP'].values


tfile = wrkdir + 'e5.oper.an.ml.0_5_0_0_0_t.regn320sc.'+ymdh+'.nc'
dsT_ERA   = xr.open_dataset( tfile )
te_ERA = dsT_ERA['T'].values


qfile = wrkdir + 'e5.oper.an.ml.0_5_0_1_0_q.regn320sc.'+ymdh+'.nc'
dsQ_ERA   = xr.open_dataset( qfile )
q_ERA = dsQ_ERA['Q'].values


ufile = wrkdir + 'e5.oper.an.ml.0_5_0_2_2_u.regn320uv.'+ymdh+'.nc'
dsU_ERA   = xr.open_dataset( ufile )
u_ERA = dsU_ERA['U'].values

vfile = wrkdir + 'e5.oper.an.ml.0_5_0_2_3_v.regn320uv.'+ymdh+'.nc'
dsV_ERA   = xr.open_dataset( vfile )
v_ERA = dsV_ERA['V'].values

wfile = wrkdir + 'e5.oper.an.ml.0_5_0_2_8_w.regn320sc.'+ymdh+'.nc'
dsW_ERA   = xr.open_dataset( wfile )
w_ERA = dsW_ERA['W'].values


print(list(dsW_ERA.variables))




# Set-up regridding machinery

In [ ]:
# Scrip file for ERA5 created by ERA5scrip.ipynb
ERA5scrip = '/glade/work/juliob/ERA5-proc/ERA5interp/grids/ERA5_640x1280_scrip.nc'

src_scrip=ERA5scrip
src_type='grid'

# ------- CAM SE ne30pg3 Scrip file
scripdir='/glade/p/cesmdata/cseg/inputdata/share/scripgrids/'
ne30scrip  = scripdir +  "ne30pg3_scrip_170611.nc"

dst_scrip=ne30scrip
dst_type='mesh'

griddir = "/glade/work/juliob/ERA5-proc/ERA5interp/grids/"
wgts_file_Con = griddir + "ERA5_ne30pg3_Conserv_wgts.nc"

# Make object for Conservative regridding from ERA5
# grid to CAM target. Scrip files need to be provided even 
# when a weight file is used
regrd, srcf, dstf = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                write_weights = False ,
                                read_weights = True ,
                                weights_file = wgts_file_Con )

In [ ]:
# Horz remap 
#########################
phis_ERA_xCAM = erg.HorzRG( aSrc = phis_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcGridkey='yx', 
                              dstGridkey='c' )


# Calculate difference in PHIS as in Williamson&Olson

$$ 
\Delta \Phi_s = \Phi^i_s - \Phi^n_s
$$

In [ ]:
#Calculate difference between phis's
Dphis = phis_ERA_xCAM - phis_CAM

# Regrid ERA5 data. 

# PS:

In [ ]:


#Horz remap. 
#########################
ps_ERA_xCAM    = erg.HorzRG( aSrc = ps_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcGridkey='tyx', 
                              dstGridkey='c' )


# Now full horizontal regridding of 4D ERA T

In [ ]:
#try function
#########################
te_ERA_xCAM    = erg.HorzRG( aSrc = te_ERA , 
                              regrd = regrd , 
                              srcField=srcf , 
                              dstField=dstf , 
                              srcGridkey='tzyx', 
                              dstGridkey='c' )


Adjust horz interpolated ERA surface pressure to account for differences in surface geopotential between remapped ERA and default CAM

$$
p^n_s= p^i_s \exp \left[ \frac{\Delta \Phi_s}{R_{dry} T_{bot}} \right ]
$$

Here $p^i_s$ is ps_ERA_xCAM

In [ ]:
#-------------------------------------------------------------------
#                    "CAM surface pressure"
#-------------------------------------------------------------------
# We don't actually have ps from CAM, so we make a guess based on
# ps_ERA_xCAM and te_bot asdescribed above.  In a sense this is a 
# vertical remapping, so we could call this variable ps_ERA_xzCAM, 
# but we'll just call it ps_CAM ...
#-------------------------------------------------------------------
nt,nz,ncol = np.shape(te_ERA_xCAM)
L_bot = nz-1

te_bot = te_ERA_xCAM[:,L_bot,:]
ps_CAM = np.zeros( (nt , ncol) )
for i in np.arange( nt ):
    ps_CAM[i,:] = ps_ERA_xCAM[i,:] * np.exp( Dphis / (Rgas*te_bot[i,:]) )

In [ ]:
#-----------------------------------------------------------------------------------------------
# Now we creat full 4(3)D pressure fields on the ERA vertical grid. These are "CAM pressures" 
# as they are based on the "CAM surface pressure" derived above.  These are used for 
# vertical interpolation below.  I'm not sure about this.  The WO2015 document seems to be
# instructing this ... but you could also use ERA surface press on xCAM.
#-----------------------------------------------------------------------------------------------
p_00 = 1.0 # a_model and a_half appear to include * P_00 in their defintion so set to 1.0 here
amid=dsT_ERA['a_model'].values 
bmid=dsT_ERA['b_model'].values
aint=dsT_ERA['a_half'].values 
bint=dsT_ERA['b_half'].values

pmid_CAM_zERA, pint_CAM_zERA, delp_CAM_zERA \
    = MkP.Pressure (am=amid ,
                    bm=bmid ,
                    ai=aint ,
                    bi=bint ,
                    ps=ps_CAM ,
                    p_00=p_00 )


In [ ]:
# Read in CAM L58 vertical grid
vCAMfile = '/glade/work/juliob/ERA5-proc/CAM-grids/Vertical/'+\
        'GRID_48_taperstart10km_lowtop_BL10_v3p1_beta1p75.nc'
vCAM=xr.open_dataset( vCAMfile )

p_00=100_000.0
amid_CAM = vCAM['hyam'].values
bmid_CAM = vCAM['hybm'].values
aint_CAM = vCAM['hyai'].values
bint_CAM = vCAM['hybi'].values
nz_CAM = np.shape( amid_CAM )[0]

pmid_CAM,pint_CAM,delp_CAM \
    = MkP.Pressure (am=amid_CAM ,
                    bm=bmid_CAM ,
                    ai=aint_CAM ,
                    bi=bint_CAM ,
                    ps=ps_CAM ,
                    p_00=p_00 )


In [ ]:
# Log-pressure is preferred for vertical interpolation
# per Williamson&Olson
#-----------------------------------------------------------
lnpint_CAM = -7_000. * np.log( pint_CAM / p_00 )
lnpmid_CAM = -7_000. * np.log( pmid_CAM / p_00 )
lnpmid_CAM_zERA = -7_000. * np.log( pmid_CAM_zERA /p_00 )


te_ERA_xzCAM = vrg.VertRG( 
        a_x  = te_ERA_xCAM ,
        zSrc = lnpmid_CAM_zERA ,
        zDst = lnpmid_CAM ,
        Gridkey ='tzc' ,
        kind = 'quadratic' 
                )
        

In [ ]:
plt.plot(   te_ERA_xCAM[0,:,20000] , lnpmid_CAM_zERA[0,:,20000] )
plt.plot(   te_ERA_xzCAM[0,:,20000] , lnpmid_CAM[0,:,20000] )
#plt.plot(   too2[0,:,20000] , lnpmid_CAM[0,:,20000] )
